# Emotion Detection

In [6]:
import pathlib

# Anzahl der Testdaten und Klassen ausgeben
data_train = pathlib.Path("./emotionen/train")
image_count = len(list(data_train.glob('*/*.png')))
features = [item.name for item in data_train.glob('*')]

print("Anzahl Testdaten:", image_count)
print("Anzahl Klassen:", len(features))

Anzahl Testdaten: 7902
Anzahl Klassen: 3


In [7]:
# Anzahl der Testdaten und Klassen ausgeben
data_test = pathlib.Path("./emotionen/test")
image_count = len(list(data_test.glob('*/*.png')))
features = [item.name for item in data_test.glob('*')]

print("Anzahl Testdaten:", image_count)
print("Anzahl Klassen:", len(features))

Anzahl Testdaten: 7178
Anzahl Klassen: 7


In [8]:
from tensorflow import keras

BATCH_SIZE = 32
IMG_HEIGHT = 224
IMG_WIDTH = 224

image_generator = keras.preprocessing.image.ImageDataGenerator(rescale=(1./255))

x_train = image_generator.flow_from_directory(directory= str(data_train),
                                              batch_size= BATCH_SIZE,
                                              shuffle = True,
                                              target_size = (IMG_HEIGHT, IMG_WIDTH),
                                              classes = features
                                             )
val_data = image_generator.flow_from_directory(directory= str(data_train),
                                              batch_size= BATCH_SIZE,
                                              shuffle = True,
                                              target_size = (IMG_HEIGHT, IMG_WIDTH),
                                              classes = features
                                             )

x_test = image_generator.flow_from_directory(directory= str(data_test),
                                              batch_size= BATCH_SIZE,
                                              shuffle = True,
                                              target_size = (IMG_HEIGHT, IMG_WIDTH),
                                              classes = features
                                             )

2023-04-26 05:44:42.468013: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-04-26 05:44:42.717859: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-04-26 05:44:42.725153: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-26 05:44:45.480780: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


Found 7902 images belonging to 7 classes.
Found 7902 images belonging to 7 classes.
Found 7178 images belonging to 7 classes.


In [9]:
# create the base model
IMG_SIZE = 224
IMG_SHAPE = (IMG_SIZE, IMG_SIZE) + (3,)
base_model = keras.applications.MobileNetV2(input_shape = IMG_SHAPE)

14536120/14536120 [==============================] - 1s 0us/step


In [11]:
base_model.trainable = False
base_model.summary()

Model: "mobilenetv2_1.00_224"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 Conv1 (Conv2D)                 (None, 112, 112, 32  864         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 bn_Conv1 (BatchNormalization)  (None, 112, 112, 32  128         ['Conv1[0][0]']                  
                                )                                              

In [12]:
input = base_model.layers[0].input
output = base_model.layers[-2].output

In [14]:
output = keras.layers.Dense(128)(output)
output = keras.layers.Activation('relu')(output)
output = keras.layers.Dense(64)(output)
output = keras.layers.Activation('relu')(output)
output = keras.layers.Dense(7, activation= 'softmax')(output)

output

<KerasTensor: shape=(None, 7) dtype=float32 (created by layer 'dense_5')>

In [15]:
# create the final model
model = keras.Model(inputs = input, outputs = output)
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 Conv1 (Conv2D)                 (None, 112, 112, 32  864         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 bn_Conv1 (BatchNormalization)  (None, 112, 112, 32  128         ['Conv1[0][0]']                  
                                )                                                             

In [16]:
model.compile(loss = 'categorical_crossentropy', optimizer ='adam', metrics =['accuracy'])

In [17]:
# train the model
history = model.fit(x=x_train, epochs=50, validation_data=val_data)

Epoch 1/50


2023-04-26 05:57:15.773600: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]
2023-04-26 05:57:22.191605: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 154140672 exceeds 10% of free system memory.
2023-04-26 05:57:22.252412: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 156905472 exceeds 10% of free system memory.


  1/247 [..............................] - ETA: 28:33 - loss: 1.9414 - accuracy: 0.0625

2023-04-26 05:57:22.935617: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 154140672 exceeds 10% of free system memory.
2023-04-26 05:57:23.010466: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 156905472 exceeds 10% of free system memory.


  2/247 [..............................] - ETA: 3:46 - loss: 1.9345 - accuracy: 0.1719 

2023-04-26 05:57:23.854573: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 154140672 exceeds 10% of free system memory.


104/247 [===========>..................] - ETA: 2:06 - loss: 1.1617 - accuracy: 0.4811

KeyboardInterrupt: 